# Shop Class - Pure Python Usage

This notebook demonstrates how to use the Shop class as a **normal Python class**.

**Key Point**: No smpub required! This is just regular Python code.

The Shop class uses smartswitch for:
- Automatic parameter validation (PydanticPlugin)
- Method dispatch
- Logging (optional)

But it works perfectly fine as a standalone Python class.

## Setup

Import the Shop class and create an instance.

In [ ]:
from sample_shop import Shop

# Create shop instance - this is just a normal Python object!
shop = Shop()

print("✅ Shop instance created")
print(f"Shop has: {', '.join([attr for attr in dir(shop) if not attr.startswith('_')])}")

## 1. Managing Article Types

Article types are categories like "electronics", "books", "clothing".

In [ ]:
# Add article types
result = shop.types.add("electronics", "Electronic devices")
print(f"✅ {result['message']}")

result = shop.types.add("books", "Books and publications")
print(f"✅ {result['message']}")

result = shop.types.add("clothing", "Clothing items")
print(f"✅ {result['message']}")

In [ ]:
# List all article types
result = shop.types.list()

print(f"\nFound {result['count']} article types:")
for t in result['types']:
    print(f"  {t['id']}. {t['name']}: {t['description']}")

## 2. Managing Articles

Articles are products with code, description, and price.

**Notice**: No manual validation needed! PydanticPlugin validates automatically:
- `price` must be > 0 (thanks to `Field(gt=0)`)
- All types are checked automatically

In [ ]:
# Add electronics
result = shop.articles.add(
    article_type_id=1,
    code="LAPTOP001",
    description="MacBook Pro 16\"",
    price=2499.00
)
print(f"✅ {result['message']}")

result = shop.articles.add(
    article_type_id=1,
    code="PHONE001",
    description="iPhone 15 Pro",
    price=1199.00
)
print(f"✅ {result['message']}")

# Add books
result = shop.articles.add(
    article_type_id=2,
    code="BOOK001",
    description="Python Programming",
    price=49.99
)
print(f"✅ {result['message']}")

In [ ]:
# List all articles
result = shop.articles.list()

print(f"\nFound {result['count']} articles:")
for article in result['articles']:
    print(f"  {article['id']}. {article['code']}: {article['description']}")
    print(f"     Type: {article['type']}, Price: ${article['price']:.2f}")

### Automatic Validation Example

Let's try to add an article with invalid price (negative).

PydanticPlugin will catch this automatically!

In [ ]:
try:
    # This will fail because price must be > 0
    result = shop.articles.add(
        article_type_id=1,
        code="INVALID001",
        description="Invalid article",
        price=-10.00  # ❌ Invalid!
    )
except Exception as e:
    print(f"❌ Validation failed (as expected): {e}")
    print("\n💡 This validation happened automatically thanks to PydanticPlugin!")
    print("   No manual 'if price <= 0' check needed in the code.")

## 3. Managing Purchases

Track purchases with quantity and automatic total calculation.

In [ ]:
# Purchase 2 laptops
result = shop.purchases.add(article_id=1, quantity=2)
print(f"✅ {result['message']}")
print(f"   Total: ${result['total']:.2f}")

# Purchase 5 books
result = shop.purchases.add(article_id=3, quantity=5)
print(f"\n✅ {result['message']}")
print(f"   Total: ${result['total']:.2f}")

# Purchase 1 phone
result = shop.purchases.add(article_id=2, quantity=1)
print(f"\n✅ {result['message']}")
print(f"   Total: ${result['total']:.2f}")

In [ ]:
# List all purchases
result = shop.purchases.list()

print(f"\nFound {result['count']} purchases:")
for purchase in result['purchases']:
    print(f"  {purchase['id']}. {purchase['quantity']}x {purchase['code']}")
    print(f"     Unit price: ${purchase['unit_price']:.2f}, Total: ${purchase['total']:.2f}")
    print(f"     Date: {purchase['purchase_date']}")

print(f"\n💰 Grand Total: ${result['grand_total']:.2f}")

## 4. Statistics

Get aggregated purchase statistics.

In [ ]:
result = shop.purchases.statistics()

print("📊 Purchase Statistics")
print("=" * 50)
print(f"Total purchases: {result['total_purchases']}")
print(f"Total revenue: ${result['total_revenue']:.2f}")
print("\nTop articles:")
for article in result['top_articles']:
    print(f"  • {article['code']}: {article['description']}")
    print(f"    Quantity: {article['total_quantity']}, Value: ${article['total_value']:.2f}")

## 5. Using Convenience Methods

The Shop class provides convenience methods that delegate to table managers.

In [ ]:
# Add type using convenience method
result = shop.add_type("toys", "Toys and games")
print(f"✅ {result['message']}")

# Get statistics using convenience method
stats = shop.get_statistics()
print(f"\n💰 Total revenue: ${stats['total_revenue']:.2f}")

## 6. Direct Access to Table Managers

You can also access table managers directly for more specific operations.

In [ ]:
# Update article price
result = shop.articles.update_price(id=1, new_price=2299.00)
print(f"✅ {result['message']}")

# Get specific article
result = shop.articles.get(id=1)
if result['success']:
    article = result['article']
    print(f"\n📱 Article: {article['code']}")
    print(f"   Description: {article['description']}")
    print(f"   Price: ${article['price']:.2f}")
    print(f"   Type: {article['type']}")

## 7. Filter Operations

List articles or purchases filtered by criteria.

In [ ]:
# List only electronics
result = shop.articles.list(article_type_id=1)

print("Electronics articles:")
for article in result['articles']:
    print(f"  • {article['code']}: {article['description']} (${article['price']:.2f})")

In [ ]:
# List purchases for a specific article
result = shop.purchases.list(article_id=1)

print("Purchases for article 1:")
for purchase in result['purchases']:
    print(f"  • {purchase['quantity']}x {purchase['code']} = ${purchase['total']:.2f}")

## Summary

### What We've Demonstrated

1. ✅ **Shop is a normal Python class** - no smpub needed
2. ✅ **Automatic validation** - PydanticPlugin validates parameters
3. ✅ **Clean API** - simple method calls, no boilerplate
4. ✅ **Flexible usage** - direct access or convenience methods

### Key Insight

```python
# This is all you need - normal Python!
shop = Shop()
shop.articles.add(type_id=1, code="ABC", desc="...", price=99.99)
```

**smpub is optional** - it just publishes this same class for CLI/HTTP access.

### Code Savings

Without smartswitch PydanticPlugin, each method would need ~10 lines of validation code:

```python
# Manual validation (NOT needed with smartswitch!)
if not isinstance(price, (int, float)):
    return {"error": "price must be number"}
if price <= 0:
    return {"error": "price must be > 0"}
# ... 8 more lines for other parameters
```

With smartswitch:

```python
# Just type hints - validation automatic!
def add(self, price: Annotated[float, Field(gt=0)]):
    # Business logic directly
```

**See [PLUGINS_EXPLANATION.md](PLUGINS_EXPLANATION.md) for more details.**